## `起源及致谢原作`

因为编写内部培训材料的需要，近期又重读了之前收藏的一篇博客《进入NLP 世界的最佳桥梁：写给所有人的自然语言处理与深度学习入门指南》
https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html ，作者是东京工作的台湾数据科学家李孟，仍然颇感获益。
由于这毕竟只是一篇博客，对其中涉及的方法实现过程写得比较简略，加之作者是台湾人的缘故，语言上也有点别扭。
因此决定以该博客为蓝本，内容加以丰富和完善，写一个自然语言处理（NLP）基础知识分享系列。目的其一作为自己的知识沉淀，其二供感兴趣的初学者学习和python实践。

依据维基百科，NLP 的定义为：自然语言处理（NLP）是计算机科学以及人工智能的子领域，专注在如何让计算机处理并分析大量（人类的）自然语言数据。NLP 常见的挑战有语音辨识、自然语言理解、机器翻译以及自然语言的生成。 
我们将围绕识别网络世界里的假新闻这个挑战场景，学习如何利用NLP知识进行工作，将使用来自Kaggle的公开数据集WSDM - Fake News Classification，
https://www.kaggle.com/datasets/wsdmcup/wsdm-fake-news-classification?select=train.csv 。 
如果没有Kaggle账号又觉得注册繁琐，也可以从本人的百度网盘中下载，链接：https://pan.baidu.com/s/18iiaF2n4z2Jf6iQN9dHANA 
提取码：w7wz 

我们先来看一下数据集的样子。

In [2]:
TRAIN_CSV_PATH = "./train.csv" 
 
import pandas as pd 
train = pd . read_csv ( TRAIN_CSV_PATH , index_col = 0 ) 
train . iloc [:,2: ] . head ( 5 ) 

,title1_zh,title2_zh,title1_en,title2_en,label
id,,,,,
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
2,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated
9,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed


训练数据集（Training Set）约有32 万笔数据、测试数据集（Test Set）则约为8 万笔。从上面的数据示例可以看出： 
 
第一列 title1_zh 代表的是「已知假新闻」 A 的中文标题，例如： 
9 用大蒜鉴别地沟油的方法,怎么鉴别地沟油 
而第二列 title2_zh 则是一笔新的新闻B 的中文标题，我们还不知道它的真伪，例如： 
吃了30年食用油才知道，一片大蒜轻松鉴别地沟油 
 
要判断第二列中的新闻标题是否为真，我们可以把它跟已知的第一篇假新闻做比较，分为3 个类别： 
•	unrelated：B 跟A 没有关系 
•	agreed：B 同意A 的叙述 
•	disagreed：B 不同意A 的叙述 
 
第五列的label 标记了比较的结果： 
如果新闻B 同意假新闻A 的叙述的话，我们可以将B 也视为一个假新闻； 
而如果B 不同意假新闻A 的叙述的话，我们可以放心地将B 新闻释出给一般大众查看； 
如果B 与A 无关的话，可以考虑再进一步处理B。 
 
在上面最后一行里，最右边的 label 列为agreed，代表B 同意A 的叙述，则我们可以判定B 也是假新闻。 
 
第三、 四 列则为新闻标题的英文翻译。而因为该翻译为机器翻译，不一定能100% 正确反映本来中文新闻想表达的意思，因此接下来的文章会忽视这两个列，只使用简体中文的新闻标题来训练NLP 模型。 
 
---

好了，就先到这儿。